In [113]:
import pandas as pd
import numpy as np
import math as m
import DataHandling

In [114]:
roadId = input('Please write the Road ID: ')



Please write the Road ID: N1


In [115]:
roads = DataHandling.readRoads(roadId)
bridges = DataHandling.readBridges(roadId)
widths = DataHandling.readWidths(roadId)
traffic = DataHandling.readTraffic(roadId)
roadnames = DataHandling.readBridges('').drop_duplicates('road', keep = 'first')['road'].tolist()

In [116]:
def formatWidthData(dataframe = widths):
    dataframe = dataframe.drop('width', 1)
    dataframe = dataframe.drop('roadId', 1)
    dataframe = dataframe.sort_values(by=['road','startChainage']).reset_index()
    dataframe = dataframe.drop('index', 1)
    return dataframe

In [117]:
def formatBridgeData(dataframe = bridges):
    dataframenoduplicates = dataframe.drop_duplicates('LRPName', keep = 'first')
    dataframemodified = dataframenoduplicates.ix[:,:10]
    dataframemodified = dataframemodified.drop('km', 1)
    dataframemodified = dataframemodified.drop('structureNr', 1)
    dataframemodified = dataframemodified.drop('type', 1)
    dataframemodified = dataframemodified.drop('roadName', 1)
    dataframemodified = dataframemodified.drop('length', 1)
    dataframemodified = dataframemodified.drop('name', 1)
    dataframemodified = dataframemodified.sort_values(by=['road','chainage']).reset_index()
    dataframemodified = dataframemodified.drop('index', 1)
    return dataframemodified

In [118]:
def formatTrafficData(dataframe = traffic, sideofroad = 'both'):
    dataframe = dataframe.drop('name',1)
    dataframe = dataframe.drop('start_lrp',1)
    dataframe = dataframe.drop('start_offset',1)
    dataframe = dataframe.drop('end_lrp',1)
    dataframe = dataframe.drop('end_offset',1)
    dataframe = dataframe.drop('length',1)
    dataframe = dataframe.reset_index()
    dataframe = dataframe.drop('index',1)
    dataframe.columns = dataframe.columns.str.replace('start_chainage','startChainage')
    dataframe.columns = dataframe.columns.str.replace('end_chainage','endChainage')
    dataframe = dataframe.sort_values(by = ['road','startChainage','linkNo'] ).reset_index()
    dataframe = dataframe.drop('index', 1)
    if sideofroad == 'both':
        dataframe = dataframe.groupby(['road','startChainage'],as_index=False).sum()
    return dataframe

In [119]:
def mergeFiles(trafficData=formatTrafficData(), bridgeData = formatBridgeData(), widthData = formatWidthData(), roadside = 'both'):
    indexesMergedData = widthData['startChainage'].searchsorted(bridgeData['chainage'], side = 'right')
    indexesMergedData =indexesMergedData.tolist()
    addedNrLanes = bridgeData
    list_a = []
    for i in indexesMergedData:
        list_a.append(widthData['nrLanes'][int(i)-1])
    addedNrLanes['nrLanes']=list_a
    bridgeData = addedNrLanes
    addedTrafficData = bridgeData
    for title in trafficData.ix[:,3:].columns:
        indexesMergedData = trafficData['startChainage'].searchsorted(bridgeData['chainage'], side = 'right')
        indexesMergedData =indexesMergedData.tolist()
        list_b=[]
        for i in indexesMergedData:
            list_b.append(trafficData[title][int(i)-1])
        addedTrafficData[title]=list_b
    return addedTrafficData

In [120]:
def calculateLaneTrafficDensity(dataframe = mergeFiles()):
    for title in dataframe.ix[:,5:].columns:
        dataframe[title] = dataframe[title]/dataframe['nrLanes']
    return dataframe

In [121]:
#Truck capacity is max capacity of type of trucks in tonnes
def calculateVulnerability(dataframe = calculateLaneTrafficDensity(), truckcapacity = [15,10,5], socialcarcapacity = [45,30,15,6,4,2,2,1,2,1], conditionlikelihood = [0.05,0.10,0.15,0.20]):
    vulbasematrix = dataframe.ix[:,:4]
    vulbasematrix.loc[vulbasematrix.condition == 'A', 'BridgeFailureLikelihood'] = conditionlikelihood[0]
    vulbasematrix.loc[vulbasematrix.condition == 'A', 'NumberOfABridges'] = 1
    vulbasematrix.loc[vulbasematrix.condition != 'A', 'NumberOfABridges'] = 0
    vulbasematrix.loc[vulbasematrix.condition == 'B', 'NumberOfBBridges'] = 1
    vulbasematrix.loc[vulbasematrix.condition != 'B', 'NumberOfBBridges'] = 0
    vulbasematrix.loc[vulbasematrix.condition == 'C', 'NumberOfCBridges'] = 1
    vulbasematrix.loc[vulbasematrix.condition != 'C', 'NumberOfCBridges'] = 0
    vulbasematrix.loc[vulbasematrix.condition == 'D', 'NumberOfDBridges'] = 1
    vulbasematrix.loc[vulbasematrix.condition != 'D', 'NumberOfDBridges'] = 0
    vulbasematrix.loc[vulbasematrix.condition == 'B', 'BridgeFailureLikelihood'] = conditionlikelihood[1]
    vulbasematrix.loc[vulbasematrix.condition == 'C', 'BridgeFailureLikelihood'] = conditionlikelihood[2]
    vulbasematrix.loc[vulbasematrix.condition == 'D', 'BridgeFailureLikelihood'] = conditionlikelihood[3]
    cargotitles = ['heavyTruck', 'mediumTruck', 'smallTruck']
    socialtitles = ['largeBus','mediumBus','microBus','utility','car','autoRickshaw','motorcycle','bicycle','cycleRickshaw','cart']
    vulbasematrix['TrafficEconomicVulnerability'] = 0
    for i in range(len(cargotitles)):
        vulbasematrix['TrafficEconomicVulnerability'] =  vulbasematrix['TrafficEconomicVulnerability']+dataframe[cargotitles[i]]*truckcapacity[i]
    vulbasematrix['TotalEconomicVulnerability'] = vulbasematrix['TrafficEconomicVulnerability']*vulbasematrix['BridgeFailureLikelihood']
    vulbasematrix['TrafficSocialVulnerability'] = 0
    for i in range(len(socialtitles)):
        vulbasematrix['TrafficSocialVulnerability'] =  vulbasematrix['TrafficSocialVulnerability']+dataframe[socialtitles[i]]*socialcarcapacity[i]
    vulbasematrix['TotalSocialVulnerability'] = vulbasematrix['TrafficSocialVulnerability']*vulbasematrix['BridgeFailureLikelihood']
    return vulbasematrix

In [122]:
def printRoadResults(dataframe = calculateVulnerability(),  roadId = roadId, root = 'vulnerability'):
    dataframe.to_csv(root+roadId+'.csv')


In [123]:
def calculatevulnerabilitysegment(dataframe = calculateVulnerability(), numberSegments=100):
    groupLevel=int(len(calculateVulnerability())/numberSegments)
    global bridges
    londictionary = dict(zip(bridges['LRPName'], bridges['lon']))
    latdictionary = dict(zip(bridges['LRPName'], bridges['lat']))
    temp = dataframe.groupby(['road',np.floor(dataframe.index/groupLevel)], as_index = False).sum()
    result = dataframe.groupby(['road',np.floor(dataframe.index/groupLevel)], as_index = False)
    bridgereliability = result.apply(lambda x: np.product(1-x['BridgeFailureLikelihood'])).reset_index(drop=True)
    lista = []
    for i in bridgereliability:
        lista.append(i.max())
    result = result.max()
    result['NumberOfABridges'] = temp['NumberOfABridges']
    result['NumberOfBBridges'] = temp['NumberOfBBridges']
    result['NumberOfCBridges'] = temp['NumberOfCBridges']
    result['NumberOfDBridges'] = temp['NumberOfDBridges']
    result['BridgeFailureLikelihood'] = 1-pd.DataFrame(lista)
    result['TotalSocialVulnerability'] = result['TrafficSocialVulnerability']*result['BridgeFailureLikelihood']
    result['TotalEconomicVulnerability'] = result['TrafficEconomicVulnerability']*result['BridgeFailureLikelihood']
    result = result.drop('condition',1)
    result['Latitude']=result['LRPName']
    result['Longitude']=result['LRPName']
    result = result.replace({"Latitude": latdictionary})
    result = result.replace({"Longitude": londictionary})
    return  result

In [124]:
def calculatetrafficpersegment(dataframe = calculateLaneTrafficDensity(), groupLevel=int(len(calculateLaneTrafficDensity())/100)):
    result = dataframe.groupby(['road',np.floor(dataframe.index/groupLevel)], as_index = False).max()
    result = result.drop('condition',1)
    return result
    

In [129]:
printRoadResults(calculatetrafficpersegment(), root='trafficpersegment')
printRoadResults(calculatevulnerabilitysegment(), root='vulnerabilitypersegment')

In [130]:
calculatevulnerabilitysegment()

,road,LRPName,chainage,BridgeFailureLikelihood,NumberOfABridges,NumberOfBBridges,NumberOfCBridges,NumberOfDBridges,TrafficEconomicVulnerability,TotalEconomicVulnerability,TrafficSocialVulnerability,TotalSocialVulnerability,Latitude,Longitude
0,N1,LRP010c,10.897,0.303597,5.0,1.0,0.0,0.0,33060.000000,10036.921986,135816.500000,41233.503172,23.699667,90.530722
1,N1,LRP013b,12.688,0.303597,5.0,1.0,0.0,0.0,44080.000000,13382.562648,181088.666667,54978.004229,23.685362,90.551359
2,N1,LRP018c,18.093,0.303597,4.0,1.0,1.0,0.0,33060.000000,10036.921986,135816.500000,41233.503172,23.654000,90.577278
3,N1,LRP021c,21.184,0.303597,2.0,3.0,1.0,0.0,33060.000000,10036.921986,135816.500000,41233.503172,23.634167,90.596889
4,N1,LRP023d,23.081,0.303597,4.0,2.0,0.0,0.0,30680.000000,9314.360754,68393.500000,20764.072106,23.613587,90.610903
5,N1,LRP028b,28.315,0.303597,6.0,0.0,0.0,0.0,30680.000000,9314.360754,68393.500000,20764.072106,23.577916,90.641972
6,N1,LRP032b,31.472,0.303597,6.0,0.0,0.0,0.0,30680.000000,9314.360754,68393.500000,20764.072106,23.555708,90.659741
7,N1,LRP034b,33.591,0.303597,6.0,0.0,0.0,0.0,30680.000000,9314.360754,68393.500000,20764.072106,23.547306,90.669417
8,N1,LRP037a,37.010,0.303597,5.0,0.0,1.0,0.0,30680.000000,9314.360754,68393.500000,20764.072106,23.529861,90.695139
9,N1,LRP049a,48.384,0.303597,1.0,1.0,4.0,0.0,29157.500000,8852.134083,55402.000000,16819.889651,23.530944,90.804194
